In [129]:
import pandas as pd
pd.options.display.max_columns = 99

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

In [130]:
df = pd.read_csv(r'C:\Users\Dask\Documents\Code\dsi\capstone_final\Data\stats_data\combined_liverpool_stats.csv')

In [131]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,HomeWin
0,2005-08-20,Liverpool,Sunderland,1.0,0.0,H,1.0,0.0,H,B Knight,15.0,2.0,6.0,0.0,8.0,20.0,5.0,5.0,2.0,3.0,0.0,1.0,1
1,2005-09-18,Liverpool,Man United,0.0,0.0,D,0.0,0.0,D,R Styles,9.0,4.0,3.0,1.0,24.0,15.0,4.0,2.0,2.0,2.0,0.0,0.0,1
2,2005-02-10,Liverpool,Chelsea,1.0,4.0,A,1.0,2.0,A,G Poll,11.0,7.0,5.0,5.0,15.0,17.0,4.0,2.0,1.0,3.0,0.0,0.0,0
3,2005-10-15,Liverpool,Blackburn,1.0,0.0,H,0.0,0.0,D,M Halsey,19.0,4.0,5.0,1.0,8.0,22.0,9.0,2.0,0.0,3.0,0.0,1.0,1
4,2005-10-29,Liverpool,West Ham,2.0,0.0,H,1.0,0.0,H,U Rennie,17.0,4.0,9.0,2.0,15.0,13.0,10.0,5.0,2.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,2021-11-12,Liverpool,Aston Villa,1.0,0.0,H,0.0,0.0,D,S Attwell,20.0,4.0,5.0,0.0,13.0,6.0,11.0,3.0,1.0,2.0,0.0,0.0,1
312,2021-12-16,Liverpool,Newcastle,3.0,1.0,H,2.0,1.0,H,M Dean,23.0,4.0,8.0,2.0,9.0,4.0,11.0,0.0,1.0,3.0,0.0,0.0,1
313,2022-01-16,Liverpool,Brentford,3.0,0.0,H,1.0,0.0,H,J Moss,27.0,6.0,13.0,1.0,6.0,6.0,9.0,0.0,0.0,1.0,0.0,0.0,1
314,2022-10-02,Liverpool,Leicester,2.0,0.0,H,1.0,0.0,H,C Kavanagh,22.0,5.0,11.0,1.0,11.0,7.0,7.0,6.0,1.0,0.0,0.0,0.0,1


In [132]:
df['FTR'] = df['FTR'].map({'H': 2, 'D': 2, 'A': 1})
df['HTR'] = df['HTR'].map({'H': 2, 'D': 2, 'A': 1})

In [133]:
df.dtypes

Date         object
HomeTeam     object
AwayTeam     object
FTHG        float64
FTAG        float64
FTR           int64
HTHG        float64
HTAG        float64
HTR           int64
Referee      object
HS          float64
AS          float64
HST         float64
AST         float64
HF          float64
AF          float64
HC          float64
AC          float64
HY          float64
AY          float64
HR          float64
AR          float64
HomeWin       int64
dtype: object

In [134]:
df['FTAG'] = df['FTAG'].astype(int)
df['FTHG'] = df['FTHG'].astype(int)
df['HTHG'] = df['HTHG'].astype(int)
df['HTAG'] = df['HTAG'].astype(int)
df['HS'] = df['HS'].astype(int)
df['AS'] = df['AS'].astype(int)
df['HST'] = df['HST'].astype(int)
df['AST'] = df['AST'].astype(int)
df['HF'] = df['HF'].astype(int)
df['AF'] = df['AF'].astype(int)
df['HC'] = df['HC'].astype(int)
df['AC'] = df['AC'].astype(int)
df['HY'] = df['HY'].astype(int)
df['AY'] = df['AY'].astype(int)
df['HR'] = df['HR'].astype(int)
df['AR'] = df['AR'].astype(int)

In [135]:
df = df.drop(columns = ['HomeTeam', 'AwayTeam', 'Referee', 'Date'])
X = df.drop(columns = ['HomeWin'])
y = df['HomeWin']
X_train, X_test, y_train, y_test = train_test_split(X_r, y, random_state = 42)
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

C:\Users\Dask\anaconda3\lib\site-packages\sklearn\utils\extmath.py:847: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Dask\anaconda3\lib\site-packages\sklearn\utils\extmath.py:689: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


In [139]:
df.isnull().sum()

FTHG       0
FTAG       0
FTR        0
HTHG       0
HTAG       0
HTR        0
HS         0
AS         0
HST        0
AST        0
HF         0
AF         0
HC         0
AC         0
HY         0
AY         0
HR         0
AR         0
HomeWin    0
dtype: int64

In [136]:
#Logistic Regression Model
logreg = LogisticRegression()
logreg.fit(X_train_sc, y_train)
logreg.score(X_train_sc, y_train), logreg.score(X_test_sc, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# K-Nearest neighbors model
knc = KNeighborsClassifier(n_neighbors=80)
knc.fit(X_train_sc, y_train)
knc.score(X_train_sc, y_train), knc.score(X_test_sc, y_test)

In [ ]:
# Decision Tree model
dtc = DecisionTreeClassifier(max_depth=5)
dtc.fit(X_train_sc, y_train)
dtc.score(X_train_sc, y_train), dtc.score(X_test_sc, y_test)

In [ ]:
# Bagging Model
bc = BaggingClassifier()
bc.fit(X_train_sc, y_train)
bc.score(X_train_sc, y_train), bc.score(X_test_sc, y_test)

In [ ]:
# Random Forest model
rfc = RandomForestClassifier(n_estimators = 30)
rfc.fit(X_train_sc, y_train)
rfc.score(X_train_sc, y_train), rfc.score(X_test_sc, y_test)

In [ ]:
# Ada boost model
abc = AdaBoostClassifier()
abc.fit(X_train_sc, y_train)
abc.score(X_train_sc, y_train), abc.score(X_test_sc, y_test)

In [ ]:
# Support vector model
svc = SVC()
svc.fit(X_train_sc, y_train)
svc.score(X_train_sc, y_train), svc.score(X_test_sc, y_test)